In [1]:
import pandas as pd

# Lecture du fichier .csv.gz (compression gzip est normalement reconnue automatiquement par pandas)
df_light = pd.read_csv('C:/Code/Projets_perso/projets/BelgiumSupplierScope/data/processed/BE_DIB_2023_clean.csv')

# Conversion sécurisée en numérique
df_light['tender_recordedbidscount'] = pd.to_numeric(df_light['tender_recordedbidscount'], errors='coerce')
df_light['tender_finalprice'] = pd.to_numeric(df_light['tender_finalprice'], errors='coerce')


C:\Users\musta\AppData\Local\Temp\ipykernel_2732\384254728.py:4: DtypeWarning: Columns (14,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_light = pd.read_csv('C:/Code/Projets_perso/projets/BelgiumSupplierScope/data/processed/BE_DIB_2023_clean.csv')


In [2]:
import pandas as pd
import numpy as np

# Supprimer les colonnes entièrement vides
df_clean = df_light.drop(columns=['corr_subm', 'corr_tax_haven'])

# Conversion des colonnes booléennes (t/f) en 1/0
bool_cols = ['bid_iswinning', 'bid_issubcontracted', 'bid_isconsortium', 'tender_isawarded']
for col in bool_cols:
    df_clean[col] = np.where(df_clean[col] == 't', 1, 0)

# Conversion finale des types
df_clean = df_clean.convert_dtypes()  # Inférence automatique des meilleurs types

In [3]:
# Colonnes géographiques potentielles
geo_columns = [
    'bidder_name','bidder_nuts', 'bidder_city', 'bidder_country',
    'buyer_name','buyer_nuts', 'buyer_city', 'buyer_country'
]

# Affichage des 5 premières lignes pour vérification
print(df_clean[geo_columns].head())

                                         bidder_name bidder_nuts  \
0                                         AT Osborne        <NA>   
1  DCE — Danish Centre for Environment and Energy...       DK042   
2  Natural Environment Research Council, Centre f...        <NA>   
3                                 The N2K Group EEIG          BE   
4  United Nations University/Centraal Bureau voor...        <NA>   

                bidder_city bidder_country  \
0                      <NA>             BE   
1                    Roende             DK   
2  Wallingford, Oxfordshire             UK   
3                  Brussels             BE   
4                     Tokyo             JP   

                                          buyer_name buyer_nuts buyer_city  \
0  Posch & Partner GmbH on behalf of the Municipa...      AT332  Innsbruck   
1        Krankenanstalten Immobiliengesellschaft mbH         AT       Graz   
2        Krankenanstalten Immobiliengesellschaft mbH         AT       Graz   
3 

#### Evaluer le taux de données géo manquantes

In [4]:
# Pourcentage de données manquantes par colonne
missing_percent = df_clean[['bidder_nuts', 'bidder_city', 'bidder_country']].isnull().mean() * 100
print(missing_percent)

bidder_nuts       61.850648
bidder_city       59.806081
bidder_country    59.586321
dtype: float64


In [5]:
missing_rows = df_clean.loc[df_clean[['bidder_nuts', 'bidder_city', 'bidder_country']].isnull().all(axis=1)]
print(f"Nombre de lignes sans info géographique du tout: {len(missing_rows)}")

Nombre de lignes sans info géographique du tout: 111827


Ces pourcentages indiquent qu’environ 61,85 % des valeurs dans la colonne bidder_nuts sont manquantes, 59,81 % dans bidder_city et 59,59 % dans bidder_country.

Cela signifie que plus de la moitié des données géographiques des fournisseurs sont absentes dans ces colonnes. Cette absence importante de données limite fortement la possibilité de réaliser une analyse géographique fiable ou une localisation précise des fournisseurs à partir de ces informations.

Il sera donc nécessaire de traiter ce problème : soit en excluant les lignes incomplètes, soit en tentant de compléter les données manquantes, ou en cherchant d’autres sources pour enrichir ces informations.

In [6]:
from geopy.geocoders import Nominatim
import pandas as pd
import time

geolocator = Nominatim(user_agent="my_geocoder")

def geocode_name(name):
    try:
        location = geolocator.geocode(name)
        if location:
            return location.latitude, location.longitude
    except:
        return None, None
    return None, None

# Exemple sur les 10 premiers acheteurs
sample_buyers = df_clean['buyer_name'].dropna().unique()[:10]
results = []

for name in sample_buyers:
    lat, lon = geocode_name(name)
    results.append({'name': name, 'latitude': lat, 'longitude': lon})
    time.sleep(1)  # Respecter les limites de l'API

df_geo = pd.DataFrame(results)
print(df_geo)

                                                name   latitude  longitude
0  Posch & Partner GmbH on behalf of the Municipa...        NaN        NaN
1        Krankenanstalten Immobiliengesellschaft mbH        NaN        NaN
2  Vlaams Centrum voor Agro- en Visserijmarketing...        NaN        NaN
3  Waterwegen en Zeekanaal NV, afdeling Bovenschelde        NaN        NaN
4                       Banque nationale de Belgique  50.848852   4.359759
5            AUTONOOM GEMEENTEBEDRIJF HEUSDEN-ZOLDER        NaN        NaN
6                              Commune de Morlanwelz        NaN        NaN
7  Agentschap voor Geografische Informatie Vlaand...        NaN        NaN
8                                   Vlaams Parlement  50.847324   4.367493
9                                        Infrabel SA  50.611238   3.405090


---

## Peut-on exploiter les noms pour trouver des coordonnées géographiques ?

### 1. **Oui, mais avec des limites**

Il est possible d’utiliser les noms d’acheteurs (`buyer_name`) et de fournisseurs (`bidder_name`) pour tenter de retrouver des coordonnées géographiques, **mais ce n’est ni automatique ni garanti**.  
Voici comment cela fonctionne et ce à quoi il faut s’attendre :

---

### 2. **Comment faire concrètement ?**

#### a) **Recherche manuelle ou semi-automatisée**
- Pour chaque nom, on peut utiliser des API ou des moteurs de recherche (Google, Bing, OpenCorporates, LinkedIn, etc.) pour tenter de retrouver l’adresse ou la localisation de l’entité.
- Si le nom est très spécifique et unique (ex : « Banque nationale de Belgique », « Vlaams Parlement »), la localisation est généralement facile à trouver.
- Pour des noms génériques ou ambigus, il peut y avoir plusieurs résultats possibles.

#### b) **Géocodage par API**
- Une fois l’adresse trouvée (ou le nom de l’organisation), on peut utiliser des services de géocodage (Google Maps API, Nominatim d’OpenStreetMap, etc.) pour obtenir les coordonnées (latitude/longitude).
- Il existe des bibliothèques Python comme `geopy` qui facilitent ce processus.

#### c) **Automatisation avec précautions**
- On peut automatiser la recherche pour les noms les plus fréquents ou les plus importants, mais il y aura des erreurs et des ambiguïtés à corriger manuellement.
- Les résultats dépendent de la qualité et de la spécificité des noms.

---

### 3. **Limites et difficultés**

- **Ambiguïté** : Plusieurs entités peuvent avoir le même nom ou des noms très proches.
- **Langues et variantes** : Les noms peuvent être écrits différemment selon la langue ou la translittération.
- **Données incomplètes** : Certains noms ne permettent pas de retrouver facilement une localisation (ex : consortiums, groupements, etc.).
- **Volume** : Avec plus de 12 000 acheteurs et 34 000 fournisseurs, le processus peut être long et nécessiter un nettoyage préalable.

---

### 4. **Exemple de démarche automatisée (Python)**

Voici un exemple de code pour géocoder des noms d’organisations :

```python
from geopy.geocoders import Nominatim
import pandas as pd
import time

geolocator = Nominatim(user_agent="my_geocoder")

def geocode_name(name):
    try:
        location = geolocator.geocode(name)
        if location:
            return location.latitude, location.longitude
    except:
        return None, None
    return None, None

# Exemple sur les 10 premiers acheteurs
sample_buyers = df_clean['buyer_name'].dropna().unique()[:10]
results = []

for name in sample_buyers:
    lat, lon = geocode_name(name)
    results.append({'name': name, 'latitude': lat, 'longitude': lon})
    time.sleep(1)  # Respecter les limites de l'API

df_geo = pd.DataFrame(results)
print(df_geo)
```

---

### 5. **Conseils pratiques**

- **Nettoyez les noms** avant de géocoder (enlever les mentions inutiles, normaliser la casse, corriger les fautes).
- **Géocodez d’abord les noms les plus fréquents** (pour maximiser la couverture).
- **Vérifiez manuellement les cas ambigus ou les échecs de géocodage**.
- **Constituez une table de correspondance** (nom → coordonnées) que vous pourrez réutiliser.

---

### 6. **Conclusion**

**Oui, il est possible d’exploiter les noms pour retrouver des coordonnées géographiques.**  
Cela demande cependant :
- Du nettoyage de données,
- Un géocodage automatisé (avec vérification humaine),
- De la patience pour traiter les cas ambigus.

C’est une bonne solution de secours quand les données structurées (codes NUTS, villes, pays) sont trop incomplètes, surtout si vous ciblez les principaux acteurs ou les analyses à grande échelle.


---
